# FOOTPRINTS

In [1]:
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from sklearn.preprocessing import normalize

In [2]:

def leer_data():
    outfile='./SOURCES/data.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data = data_original[['client_id','date','año','mes','dia','hora','mccg','quantity','amount_sol']]
data.head(2)

,client_id,date,año,mes,dia,hora,mccg,quantity,amount_sol
0,NNeQwQy9MAQ=,2016-07-15 22:23:25,2016,7,15,22,2,1,100.0
1,qFRoPHPOc/I=,2016-06-21 21:30:55,2016,6,21,21,2,1,20.0


In [3]:
mccgs = ((data.groupby(['mccg'], as_index=False, sort=True)).count())['mccg']
mccgs,mccgs[3],len(mccgs)

(0      2
 1      3
 2      4
 3      5
 4      6
 5      8
 6      9
 7     10
 8     11
 9     12
 10    13
 11    14
 12    15
 13    16
 14    17
 15    18
 16    19
 17    21
 18    22
 19    23
 20    25
 21    26
 22    27
 23    28
 Name: mccg, dtype: int64, 5, 24)

# RESULTADOS

## Unimos los resultados por cada mccg

In [4]:

path='./RESULTS' 

file='./RESULTS/U' 
footprint="%s.footprint" %(file)
individual_clusters="%s.individual_footprint.clusters" %(file)
individual_labels="%s.individual_footprint.labels" %(file)   
collective_clusters="%s.collective_footprint.clusters" %(file)
collective_labels="%s.collective_footprint.labels" %(file)


In [5]:
a = pd.read_csv(individual_labels, sep=";", header=0, index_col=False)   ## read file
b = pd.read_csv(collective_labels, sep=";", header=0, index_col=False)   ## read file

In [6]:
a = pd.merge(a, b, on=['customer_tag', 'individual_cluster'])
del b
a = a.iloc[:, :(-1*4*7*len(mccgs))]

In [7]:
b = pd.read_csv(footprint, sep=",", header=0)   ## read file
b.rename(columns={'customer_id':'customer_tag'}, inplace=True)  

In [8]:
a = pd.merge(b, a, on=['customer_tag','year', 'week'])
del b
b = a.iloc[:, :(-1*4*7*len(mccgs)-1)]

In [9]:
a = pd.concat([b, a[['collective_cluster']]], axis=1)
del b
a[['year','week','profile_id','size','individual_cluster','collective_cluster']] = a[['year','week','profile_id','size','individual_cluster','collective_cluster']].astype(int)

In [10]:
path_res='%s/Results.csv' %(path)
a.to_csv(path_res,index=False)
print('Done')

Done
